# InfVis project work
---
This Python Script downloads an up-to-date dataset for **Covid-19**, gives first insights on the data and exports the data to the data folder.

In [16]:
%matplotlib inline
import pandas as pd
import numpy as np

import io               # file operations
import json

import ssl              # secure client-server connection
import requests         # html-requests

In [4]:
# Make a get request to get the latest position of the international space station from the opennotify api.
response = requests.get("http://api.open-notify.org/iss-now.json")
# Print the status code of the response.
print(response.status_code)

200


In [9]:
url = 'https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query?where=1%3D1&outFields=Bundesland,AnzahlFall,AnzahlTodesfall,NeuerFall,Datenstand,AnzahlGenesen,IstErkrankungsbeginn,NeuerTodesfall,ObjectId,Landkreis,Meldedatum,Refdatum,NeuGenesen,IdLandkreis,Altersgruppe,IdBundesland&outSR=4326&f=json'

In [10]:
r = requests.get(url)
print(r.status_code)

200


In [13]:
r.headers

{'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '56384', 'Connection': 'keep-alive', 'Cache-Control': 'public, max-age=3600, s-maxage=3600', 'Content-Encoding': 'gzip', 'Last-Modified': 'Thu, 09 Jul 2020 01:16:28 GMT', 'ETag': 'sd139_-1138325323', 'Server': 'Microsoft-IIS/10.0', 'x-esri-query-request-units': '2', 'X-ArcGIS-Correlation-Id': '|dcfbf625ad3bfa438c5925c9d8c613d2.e2cee2c0_', 'X-ArcGIS-Instance': 'MTSDS_Web_IN_23', 'X-Powered-By': 'ASP.NET', 'Access-Control-Allow-Origin': '*', 'Date': 'Thu, 09 Jul 2020 19:59:03 GMT', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 784dd167d622737126ee2d76985e7d3c.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'FRA2-C2', 'X-Amz-Cf-Id': '-sLZsBNnFZG7SVH7HS7lRYEXbze-kE-s3VUVYqoBhGGBn7_req_xmg=='}

In [17]:
data = json.loads(r.text)
print(type(data))

<class 'dict'>


In [22]:
for item in data['fields']:
    print(item['name'])

Bundesland
AnzahlFall
AnzahlTodesfall
NeuerFall
Datenstand
AnzahlGenesen
IstErkrankungsbeginn
NeuerTodesfall
ObjectId
Landkreis
Meldedatum
Refdatum
NeuGenesen
IdLandkreis
Altersgruppe
IdBundesland


In [26]:
totalCases = 0
totalEntries = 0
for entry in data['features']:
    totalCases += entry['attributes']['AnzahlFall']
    totalEntries += 1
print(totalCases, entries)

7127 5000


Testing commulated data for comparison with numbers from the internet

In [39]:
shDeaths = 0
shRecovered = 0
shEntries = 0
for entry in data['features']:
    if entry['attributes']['Bundesland'] == 'Schleswig-Holstein':
        shDeaths += entry['attributes']['AnzahlTodesfall']
        shRecovered += entry['attributes']['AnzahlGenesen']
        shEntries += 1
print('Deaths:', shDeaths, 'Recovered:', shRecovered, 'Entries:', shEntries)

Deaths: 153 Recovered: 3005 Entries: 2882
